In [15]:
import config 
import requests 
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import re

In [2]:
# can update this with your api key to get it to work 
url = f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={config.API_KEY}'
response1 = requests.get(url).json()

## Sorting and Cleaning Functions 

In [ ]:

def sort_data(response,response_key,delimter=None):
    response_list = []
    # makes a list of the values given in the response 
    for i,_ in enumerate(response['results']):
        response_list.append(response['results'][i][response_key])
    
    # for the keywords splits the dictinary apart, then the words
    if response_key == 'adx_keywords':
        response_string_list = str(response_list).split(delimter)
        response_string_list = str(response_string_list).split()
        return response_string_list
    
    return response_list
    

def clean_data(response_string_list):
    
    # makes a dictionary that has the count of suplicates for each string
    keyword_count = dict(Counter(response_string_list))
    
    sorted_keyword_count = dict(sorted(keyword_count.items(),key=lambda item: item[1],reverse=True))
    
    # removes keywords that are non-alphanumeric characters
    for key in sorted_keyword_count.copy():
        if re.search("^\W",key):
            sorted_keyword_count.pop(key)
        elif key == 'and' or key == 'of' or key=='the':
            sorted_keyword_count.pop(key)
                
    return sorted_keyword_count

# Popular Articles on New York Times

In [ ]:

title_response_string_list = sort_data(response1,'title')

print('Popular Titles')
for i,_ in enumerate(title_response_string_list):
    print(f'{i+1}. {title_response_string_list[i]}')

# Keywords in Top 20 Most Popular NYT Articles

In [3]:
# returns responses froim sort and clean fucntions
keyword_response_string_list = sort_data(response1,'adx_keywords',';')
sorted_keyword_count = clean_data(keyword_response_string_list)

In [ ]:
def get_top_keywords(sorted_keyword_count,top_number):
    # get last ten entries as list is sorted in reverse
    top_keywords = list(sorted_keyword_count)[:top_number]
    # remove symbols and other non-alphanumeric
    top_keywords = [re.sub(r'\W','',key) for key in top_keywords]   
    # get top counts 
    top_counts = list(sorted_keyword_count.values())[:top_number]
    
    return top_keywords,top_counts

In [31]:
# prints popular keywords
TOP_NUMBER = 10
top_keywords,top_counts = get_top_keywords(sorted_keyword_count,TOP_NUMBER)

for i,_ in enumerate(top_keywords):
    print(f'{i+1}. {top_keywords[i]} was a keyword {top_counts[i]} times in the {TOP_NUMBER} most popular NYT articles today')


In [ ]:
# makes graph of keyword occurance 
figure(figsize=(12,9), dpi=80)
bar = plt.bar(top_keywords,top_counts)
plt.xlabel('Key Words')
plt.ylabel('Occurances of Word')
plt.title(f'Most Popular Keywords in Top {TOP_NUMBER} NYT Articles Today')



plt.show()